In [241]:
'''
Plane電廠資訊
'''
from typing import Optional, Dict, List
from pathlib import Path
import requests
import json
import pandas as pd

API_KEY = 'iVz55YLEFMa23pZb5Zj095sXOXjTnErj7pZ8SdMs'

def getPlantInfo(

    file_data=None
):
    url = 'https://solar-api.hdre.energy/getPlantInfo'
    headers = {
        'Content-Type': 'application/json',
        'x-api-key': API_KEY
    }

    response = requests.get(url, headers=headers)
    return response


a = getPlantInfo()

def load_pd_from_json(data:Dict, dtypes:Optional[Dict]=None) -> Optional[pd.DataFrame]:
    if data:
        df = pd.DataFrame(data=data)
        if dtypes:
            df = df.astype(dtypes)
        df = df.replace({pd.NA: None, pd.NaT: None})
        return df
    return None

In [242]:
data = a.content

# 解码JSON数据
decoded_data = data.decode('unicode-escape')

# 转换为Python对象
parsed_data = json.loads(decoded_data)
df = pd.DataFrame(parsed_data)

plane_list = df['plantNo'].unique().tolist()
display(plane_list)

ownerName_list = df['ownerName'].unique().tolist()
ownerName_list 

['T18A17R302',
 'T19A17R302',
 'T19A28G101',
 'T19A61G101',
 'T20A17G301',
 'T20A60G101',
 'T20A62G101',
 'T20A65G101']

['泓德', '方登綠能(星泓)', '大福(星泓)', '中鈁綠能(星泓)', '仁華綠能(星泓)', '茂泓(星泓)']

In [243]:
'''
發電量資訊
'''
def getPlantHour(
    plantNo: str,
    timeTo: str,
    timeFrom: str ,
):
    url = 'https://solar-api.hdre.energy/getPlantHourly'
    headers = {
        'Content-Type': 'application/json',
        'x-api-key': API_KEY
    }
    params = {
    "plantNo": plantNo,
    "timeTo": timeTo,
    "timeFrom": timeFrom
    }

    response = requests.get(url,  params=params,headers=headers)
    return response

plant_no = "T20A65G101"
time_to = "2023-02-01 00:00:00"
time_from = "2023-01-01 00:00:00"
b = getPlantHour(plant_no,time_to,time_from)

In [244]:
date_range = pd.date_range(start=time_from, end=time_to, freq='H')

# 創建 DataFrame
df2 = pd.DataFrame({"formatted_periodTime": date_range})

# 如果需要將 timestamp 列格式化為字串，可以使用以下代碼
df2["formatted_periodTime"] = df2["formatted_periodTime"].dt.strftime("%Y/%m/%d %H:%M:%S")
df2['intervalAcKwh'] = 0.0
df2.dtypes


formatted_periodTime     object
intervalAcKwh           float64
dtype: object

In [245]:
data = b.content
decoded_data = data.decode('unicode-escape')

# 转换为Python对象
parsed_data = json.loads(decoded_data)
#df = pd.DataFrame(parsed_data)
#df['data'][0]

In [246]:
'''
兩層dataframe強轉
'''
data = b.json()
df = pd.DataFrame(data)
df1 = pd.DataFrame(df.data[0])
display(df1)

from datetime import datetime, timezone, timedelta

def convert_period_time(row):
    original_datetime = datetime.fromisoformat(row["periodTime"])
    original_datetime_utc = original_datetime.astimezone(timezone.utc)
    adjusted_datetime = original_datetime_utc.astimezone(timezone(timedelta(hours=8)))
    formatted_date_string = adjusted_datetime.strftime("%Y/%m/%d %H:%M:%S")
    return formatted_date_string

# 將函數應用到 periodTime 列
df1["formatted_periodTime"] = df1.apply(convert_period_time, axis=1)
df1.dtypes

,periodTime,intervalAcKwh,avgPower,avgRadiation,prRate
0,2023-01-01T06:00:00+08:00,0.29,0.00,0.0005,0.0274
1,2023-01-01T07:00:00+08:00,687.42,0.03,0.0305,1.0647
2,2023-01-01T08:00:00+08:00,1849.05,0.09,0.0824,1.0601
3,2023-01-01T09:00:00+08:00,3311.58,0.16,0.1538,1.0172
4,2023-01-01T10:00:00+08:00,4217.53,0.20,0.1940,1.0270
...,...,...,...,...,...
429,2023-01-31T15:00:00+08:00,7999.25,0.38,0.4583,0.8246
430,2023-01-31T16:00:00+08:00,3857.27,0.18,0.2269,0.8031
431,2023-01-31T17:00:00+08:00,636.09,0.03,0.0412,0.7294
432,2023-01-31T18:00:00+08:00,0.30,0.00,0.0000,0.0000


periodTime               object
intervalAcKwh           float64
avgPower                float64
avgRadiation            float64
prRate                  float64
formatted_periodTime     object
dtype: object

In [247]:
plantName = df['plantName'].str.split('-')[0][0]

planeCapacity = df['capacity']

df


,ownerName,plantNo,plantName,capacity,on_grid_date,city,latitude,longitude,data
0,茂泓(星泓),T20A65G101,茂泓電業-21167.9KWp,21167.9,2022-01-17,臺南市,23.272418,120.139361,"[{'periodTime': '2023-01-01T06:00:00+08:00', '..."


In [248]:
'''
import requests

url = "https://solar-api.hdre.energy/getPlantHourly"
params = {
    "plantNo": "T20A65G101",
    "timeTo": "2023-06-01 00:00:00",
    "timeFrom": "2023-05-01 00:00:00"
}

headers = {
    "accept": "application/json",
    "x-api-key": "iVz55YLEFMa23pZb5Zj095sXOXjTnErj7pZ8SdMs"
}

try:
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        # Successful request
        data = response.json()
        print(data)
    else:
        # Print error message if request was not successful
        print(f"Error: {response.status_code} - {response.text}")

except requests.RequestException as e:
    print(f"Request failed: {e}")
'''

'\nimport requests\n\nurl = "https://solar-api.hdre.energy/getPlantHourly"\nparams = {\n    "plantNo": "T20A65G101",\n    "timeTo": "2023-06-01 00:00:00",\n    "timeFrom": "2023-05-01 00:00:00"\n}\n\nheaders = {\n    "accept": "application/json",\n    "x-api-key": "iVz55YLEFMa23pZb5Zj095sXOXjTnErj7pZ8SdMs"\n}\n\ntry:\n    response = requests.get(url, params=params, headers=headers)\n\n    if response.status_code == 200:\n        # Successful request\n        data = response.json()\n        print(data)\n    else:\n        # Print error message if request was not successful\n        print(f"Error: {response.status_code} - {response.text}")\n\nexcept requests.RequestException as e:\n    print(f"Request failed: {e}")\n'

In [249]:
print("Columns in df1:", df1.columns)
print("Columns in df2:", df2.columns)

merged_df = pd.merge(df2, df1, left_on='formatted_periodTime', right_on='formatted_periodTime', how='left', suffixes=('_df2', '_df1'))
merged_df
merged_df["intervalAcKwh_df2"] = merged_df["intervalAcKwh_df1"].combine_first(merged_df["intervalAcKwh_df2"])
merged_df.drop(["intervalAcKwh_df1"], axis=1, inplace=True)
merged_df['intervalAcKwh_df2']

import plotly.express as px

fig = px.bar(merged_df, x='formatted_periodTime', y='intervalAcKwh_df2', title='小時資料')
fig.update_xaxes(title_text='Formatted Period Time')
fig.update_yaxes(title_text='發電度數kwh')
fig.show()

Columns in df1: Index(['periodTime', 'intervalAcKwh', 'avgPower', 'avgRadiation', 'prRate',
       'formatted_periodTime'],
      dtype='object')
Columns in df2: Index(['formatted_periodTime', 'intervalAcKwh'], dtype='object')


In [250]:
col = ['formatted_periodTime','intervalAcKwh_df2']
final = merged_df[col]
final = final.rename(columns={'formatted_periodTime': '時間','intervalAcKwh_df2': plantName })
final.to_csv('power_generation.csv')